# Assignment Set 2
by Zahraoruji for Deep Learing Course at Data Science Center, SBU

In [1]:
import numpy as np 
import pandas as pd
import torch
import torchvision
import torchvision.transforms as transforms

In [17]:


trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [18]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [19]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [20]:
LAMBDA = 0.009

for epoch in range(2):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
         #L1 regularization
        
        l1_regularization = torch.tensor(0).long()
        for param in net.parameters():
            l1_regularization += torch.norm(param, 1).long()
            
        loss += LAMBDA * l1_regularization
        
        
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 18.806
[1,  4000] loss: 18.856
[1,  6000] loss: 18.925
[1,  8000] loss: 18.998
[1, 10000] loss: 19.099
[1, 12000] loss: 19.207
[2,  2000] loss: 19.310
[2,  4000] loss: 19.392
[2,  6000] loss: 19.486
[2,  8000] loss: 19.588
[2, 10000] loss: 19.660
[2, 12000] loss: 19.746
Finished Training


In [21]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 61 %
Accuracy of   car : 14 %
Accuracy of  bird :  6 %
Accuracy of   cat : 12 %
Accuracy of  deer :  2 %
Accuracy of   dog : 28 %
Accuracy of  frog : 62 %
Accuracy of horse : 16 %
Accuracy of  ship : 15 %
Accuracy of truck :  9 %


In [15]:
LAMBDA = 0.009

for epoch in range(2):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
         #L1 regularization
        
        l2_regularization = torch.tensor(0).long()
        for param in net.parameters():
            l2_regularization += torch.norm(param, 1).long()
            
        loss += LAMBDA * l2_regularization
        
        
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 20.008
[1,  4000] loss: 20.113
[1,  6000] loss: 20.200
[1,  8000] loss: 20.287
[1, 10000] loss: 20.395
[1, 12000] loss: 20.516
[2,  2000] loss: 20.626
[2,  4000] loss: 20.728
[2,  6000] loss: 20.815
[2,  8000] loss: 20.880
[2, 10000] loss: 20.973
[2, 12000] loss: 21.063
Finished Training


In [16]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 35 %
Accuracy of   car : 19 %
Accuracy of  bird : 10 %
Accuracy of   cat : 12 %
Accuracy of  deer :  7 %
Accuracy of   dog : 32 %
Accuracy of  frog : 38 %
Accuracy of horse : 23 %
Accuracy of  ship : 34 %
Accuracy of truck : 14 %
